In [353]:
%load_ext autoreload
%autoreload

import networkx as nx
import networkx.algorithms as algos
from networkx.algorithms import approximation
from networkTrips import organizeTrips
from networkAlgos import networkAlgos
from timeUtils import clock, elapsed, getDateTime
from collections import Counter
from haversine import haversine
from ioUtils import loadJoblib
from pandasUtils import getRowData, getColData
from networkAlgos import networkAlgos
from edgeInfo import edgeInfo
from vertexInfo import vertexInfo
from networkCategories import categories
from place import getPlaceData
from cbsa import getCBSAData
from csa import getCSAData
from metdiv import getMetDivData
from county import getCountyData
from state import getStateData
from geocluster import geoClusters
from geoUtils import convertMetersToLat, convertLatToMeters, convertMetersToLong, convertLongToMeters
from geoclusterUtils import genCenters, genCluster, genClusters, genTripsBetweenClusters

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [474]:
#######################################################################################
# Generate Clusted Data
#######################################################################################
genData = True
if genData:
    total   = 500
    genMax  = 75
    distMax = 500
    raw  = genClusters(20, 250, latRange=[29.8, 30.2], lngRange=[49.8, 50.2], dist="gauss", maxrad=genMax)
    gc   = geoClusters(key="dummy", points=raw, distMax=distMax, debug=False)
    gc.findClusters(seedMin=2, debug=False)
    df   = genTripsBetweenClusters(n=total, gc=gc, returnDF=True)
    df["device"] = "dummy"
    
    tmpdf = loadJoblib("/Users/tgadf/Downloads/r4hIDs.p").sample(n=total, replace=True)
    from pandas import concat
    newdf = concat([df, tmp], axis=1)
else:
    df = loadJoblib("/Users/tgadf/Downloads/r4h.p")

Selected 500 randomized trips
Found Start/End for the 500 randomized trips
Converting (500, 2, 2) trips to a DataFrame


In [475]:
newdf.shape

(500, 10)

In [472]:
tmpdf = loadJoblib("/Users/tgadf/Downloads/r4hIDs.p")
from pandas import concat
concat([tmpdf,tmpdf,tmpdf], axis=1).shape

(1000, 414)

In [463]:
sample = df[list(df.columns)[10:]].sample(n=1000)
from ioUtils import saveJoblib
saveJoblib(sample, "/Users/tgadf/Downloads/r4hIDs.p")

  --> This file is 198.3kB.


In [462]:
# Show Data (if needed)
df

,device,Start,End,Duration,total_miles,inR4HTimePeriod,lat0,long0,lat1,long1,...,Geo0ROADSMajorRdID,Geo1ROADSMajorRdID,Geo0POIVisitsID,Geo0POIUniqueVisitsID,Geo1POIVisitsID,Geo1POIUniqueVisitsID,Geo0TerminalsAirportID,Geo0TerminalsAmtrakID,Geo1TerminalsAirportID,Geo1TerminalsAmtrakID
0,RWDEB97LSA1H4IZ,2017-12-23T10:01:48-10:00,2017-12-23T10:23:44-10:00,1315,14.10,Y,21.385278,-157.950076,21.312765,-158.055588,...,0,0,1853,191,0,0,0.0,0.0,0.0,0.0
1,53XC3IYW67C4O0E,2017-10-31T15:44:31-10:00,2017-10-31T17:08:54-10:00,5063,13.20,Y,21.440997,-158.184695,21.339682,-158.078461,...,0,0,6,3,1263,54,0.0,0.0,0.0,0.0
2,M4CGPR1TYHBKVJK,2017-12-24T03:23:26-10:00,2017-12-24T03:39:59-10:00,992,8.40,Y,21.293925,-157.821295,21.334003,-157.923459,...,1,0,48,20,199,97,0.0,0.0,1.0,0.0
3,RWDEB97LSA1H4IZ,2017-12-31T16:16:26-10:00,2017-12-31T16:26:32-10:00,605,3.85,Y,21.330069,-157.879447,21.333042,-157.923245,...,0,0,225,14,199,97,0.0,0.0,1.0,0.0
4,3T96FV77UYC17DJ,2017-12-22T18:02:46-10:00,2017-12-22T18:33:13-10:00,1827,7.80,Y,21.367507,-157.925737,21.327959,-157.899446,...,0,0,4,1,4,1,0.0,0.0,0.0,0.0
5,PFLXOYIQSN535UC,2017-12-25T10:05:32-10:00,2017-12-25T10:30:09-10:00,1476,12.64,Y,21.275205,-157.824777,21.328160,-157.899595,...,1,0,456,56,4,1,0.0,0.0,0.0,0.0
6,JY2R26AMISS5BG1,2017-12-28T20:27:50-10:00,2017-12-28T20:37:43-10:00,593,5.73,Y,21.278178,-157.811567,21.321931,-157.866710,...,1,0,13,4,542,78,0.0,0.0,0.0,0.0
7,BZU6BCY6DYBNOFY,2017-07-17T15:04:16-10:00,2017-07-17T15:23:10-10:00,1134,5.41,Y,21.278456,-157.811646,21.321742,-157.856744,...,1,0,13,4,840,131,0.0,0.0,0.0,0.0
8,NCGFHU5YH0N8BXA,2017-09-07T12:50:37-10:00,2017-09-07T13:01:18-10:00,640,3.30,Y,21.359389,-157.853660,21.322711,-157.856991,...,0,0,4,2,840,131,0.0,0.0,0.0,0.0
9,BZU6BCY6DYBNOFY,2017-09-05T09:20:29-10:00,2017-09-05T09:37:34-10:00,1024,4.07,Y,21.298647,-157.840693,21.322458,-157.857047,...,0,0,2012,129,840,131,0.0,0.0,0.0,0.0


In [6]:
#######################################################################################
# Cluster Geo Data (Lat, Long)
#######################################################################################
points         = df[["lat0", "long0"]]
points.columns = ["lat", "long"]
pnts           = df[["lat1", "long1"]]
pnts.columns   = ["lat", "long"]    
points         = points.append(pnts)



#######################################################################################
# Create Clusters
#######################################################################################
debug=True
gc   = geoClusters(key="dummy", points=points, distMax=300, debug=debug)
gc.findClusters(seedMin=2, debug=debug)
if debug:
    print("Found {0} clusters using {1} cells and {2} counts".format(gc.getNClusters(), gc.getNCells(), gc.getNCounts()))

    
    
#######################################################################################
# Set Nearest Clusters
#######################################################################################
if debug:
    start, cmt = clock("Finding Nearest Clusters for Start of Trips")
geoResults = df[['lat0', 'long0']].apply(gc.getNearestClusters, axis=1).values
df["geo0"] = [x[0] for x in geoResults]
if debug:
    elapsed(start, cmt)
    start, cmt = clock("Finding Nearest Clusters for End of Trips")
geoResults = df[['lat1', 'long1']].apply(gc.getNearestClusters, axis=1).values
df["geo1"] = [x[0] for x in geoResults]    
if debug:
    elapsed(start, cmt)

    
    
#######################################################################################
# Organize Trips for Network
#######################################################################################
trips = organizeTrips(df=df, gc=gc, debug=False, requireGood=False)

Current Time is Mon Nov 12, 2018 10:12:13 for Converting 2000 Points To Correct Format
Data has correct format with a (2000, 2) shape.
Current Time is Mon Nov 12, 2018 10:12:13 for Done with Converting 2000 Points To Correct Format
Process [Done with Converting 2000 Points To Correct Format] took 0 seconds.
Current Time is Mon Nov 12, 2018 10:12:13 for Finding Geohash (BitLen=8) Values from 2000 Points
Current Time is Mon Nov 12, 2018 10:12:14 for Done with Finding Geohash (BitLen=8) Values from 2000 Points
Process [Done with Finding Geohash (BitLen=8) Values from 2000 Points] took 0 seconds.
Current Time is Mon Nov 12, 2018 10:12:14 for Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame
Current Time is Mon Nov 12, 2018 10:12:14 for Done with Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame
Process [Done with Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame] took 0 seconds.
Current Time is Mon Nov 12, 2018 10:12:14 for Finding Cluster

In [340]:
class network():
    def __init__(self, directed=True, debug=False):
        self.debug = debug
        self.directed = directed
        
        self.orderedEdges    = None
        self.edgeDict        = None
        self.orderedVertices = None
        self.nodeDict        = None
        
        if self.directed is True:
            self.g = nx.DiGraph()
        else:
            self.g = nx.Graph()

        self.eInfo            = edgeInfo(self.g, self.debug)
        self.getEdges         = self.eInfo.getEdges
        self.getEdge          = self.eInfo.getEdgeData
        self.getEdgeAttrs     = self.eInfo.getAttrGroups
        self.setEdgeFeature   = self.eInfo.setEdgeFeature        
        self.getEdgeNum       = self.eInfo.getEdgeNumByName
        
        self.vInfo            = vertexInfo(self.g, self.debug)
        self.getVertices      = self.vInfo.getVertices
        self.getVertex        = self.vInfo.getVertexData
        self.getVertexByName  = self.vInfo.getVertexDataByName
        self.getVertexAttrs   = self.vInfo.getAttrGroups
        self.setVertexFeature = self.vInfo.setVertexFeature        
        self.getVertexNum     = self.vInfo.getVertexNumByName
            
    def setDebug(self, debug):
        self.debug = debug
        
    def getNetwork(self):
        return self.g
    
    
    def update(self):
        self.eInfo.orderEdges()
        self.vInfo.orderVertices()
        
            
    def flattenAttrs(self):
        self.eInfo.flattenEdgeAttrs()
        self.vInfo.flattenVertexAttrs()
        
    
    def collectAttrs(self):
        self.eInfo.collectEdgeAttrs()
        self.vInfo.collectVertexAttrs()
    
    
    ################################################################################################
    # Show Network Data
    ################################################################################################    
    def showVertices(self):
        for nodename,node in self.g.nodes_iter(data=True):
            print(nodename,'\t',node)
                
    def showEdges(self):
        for edgename,edge in self.g.adj.items():
            print(edgename,'\t',edge)
                
                

        
    ################################################################################################
    # Vertices / Nodes / Location (Initial Functions)
    ################################################################################################    
    def addVertex(self, name, attrs={}):
        self.g.add_node(u=name, attr_dict=attrs)
        if self.debug:
            print("  Added node: [{0}]".format(", ".join(names)))
                    
    def updateVertexAttrs(self, attrs):
        if not isinstance(attrs, dict):
            print("Cannot add vertex attrs because the input is not a dict")
            return
        nx.set_node_attributes(G=self.g, values=attrs, name=None)
            
        
        
    ################################################################################################
    # Edges / Trips (Initial Functions)
    ################################################################################################    
    def addEdge(self, names, attrs={}, sort=False):
        if not isinstance(names, (tuple,list,set)):
            print("Cannot add edge {0} because the names need to come in a tuple/list/set.".format(names))
            return
        if len(names) == 2:
            if sort is True:
                names = sorted([str(x) for x in names])
            else:
                names = [str(x) for x in names]
        else:
            print("Cannot add edge {0} because we need two entries in the tuple/list/set.".format(names))
            return
        
        self.g.add_edge(names[0], names[1], attr_dict=attrs)
        if self.debug:
            print("  Added edge: [{0}]".format(", ".join(names)))
            
    def updateEdgeAttrs(self, attrs):
        if not isinstance(attrs, dict):
            print("Cannot add edge attrs because the input is not a dict")
            return
        nx.set_edge_attributes(G=self.g, values=attrs)

In [399]:
class driverNetwork(network):
    def __init__(self, trips):
        network.__init__(self, directed=False, debug=False)
        
        self.categories        = categories(debug)
        self.getCategories     = self.categories.getCategories
        self.getCategory       = self.categories.getCategory
        self.getPermCategories = self.categories.getPermCategories
        self.getPermCategory   = self.categories.getPermCategory  
        self.getHomeRatioCategory = self.categories.getHomeRatioCategory
        self.getIntervalCategory = self.categories.getIntervalCategory
        
        
        if trips is not None:
            if isinstance(trips, dict):
                self.name          = trips.get('device')
                self.edgeMetrics   = trips.get('edgeMetrics')
                self.vertexMetrics = trips.get('vertexMetrics')
                self.vertexMetrics = {str(k): v for k,v in self.vertexMetrics.items()}
                self.homeMetrics   = trips.get('homeMetrics')
                print("Creating a driver network with {0} vertices and {1} edges.".format(len(self.vertexMetrics), len(self.edgeMetrics)))
            else:
                raise ValueError("Input trips must be a dictionary of edgeMetrics, vertexMetrics, and homeMetrics (optional)")
        else:
            raise ValueError("Input trips is None!")

            
    ####################################################################################
    # Create Network
    ####################################################################################
    def create(self, debug=False):
        for edgename,edgedata in self.edgeMetrics.items():
            self.addEdge(edgename, edgedata)
        self.updateVertexAttrs(self.vertexMetrics)
        self.update()
        self.flattenAttrs()
        self.collectAttrs()

        
    ####################################################################################
    # Compute Network Attributes
    ####################################################################################
    def computeNetworkAttrs(self, debug=False):
        self.netAlgos = networkAlgos()
        results = self.netAlgos.compute(self.g)
        self.nodeAttrs = results['Nodes']
        self.edgeAttrs = results['Edges']
        self.edgeAttrs['edge_weight'] = self.eInfo.getEdgeWeights().values() # add weights
        self.netAttrs  = results['Net']
        if debug:
            print("  Created {0} attributes for {1} vertices".format(self.nodeAttrs.shape[1], self.nodeAttrs.shape[0]))
            print("  Created {0} attributes for {1} edges".format(self.edgeAttrs.shape[1], self.edgeAttrs.shape[0]))
            print("  Created {0} attributes for the network".format(len(self.netAttrs)))


    ####################################################################################
    # Perform Lookup for Census Data
    ####################################################################################
    def fillCensusData(self, debug=False):
        if debug:
            print("Filling Vertex Census Data")
        verydebug=False

        censusKeys = [k for k,v in self.getVertexAttrs().items() if v == "Census"]
        getCensusData = {"CensusCbsa": getCBSAData, "CensusCsa": getCSAData, "CensusCounty": getCountyData, "CensusMetdiv": getMetDivData, "CensusPlace": getPlaceData, "CensusState": getStateData}
        for key in censusKeys:
            for vertexName in self.getVertices():
                vertex = self.getVertexByName(vertexName, 'attr')
                if verydebug:
                    print("  --> Vertex Number {0} and ID {1}".format(vertexNum, vertexID))

                value   = vertex[key]
                
                if isinstance(value, list):
                    try:
                        #mc    = value.most_common(1)
                        value = value[0][0]
                    except:
                        print("There was an error getting most common {0}".format(key))
                        value = None        
                else:
                    print("Input {0} is type {1}".format(value, type(value)))
                try:
                    lookup  = getCensusData[key](str(value))
                except:
                    lookup  = value

                self.setVertexFeature(vertexName, key, lookup)
                if verydebug is True:
                    print("\t: {0}, {1} == {2}".format(key, value, lookup))
                    
        if verydebug:
            raise ValueError("Stoppping after verydebug is True")            
            

    def fillGeospatialData(self, debug=False):
        if debug:
            print("Filling Vertex Geospatial Data")
        verydebug=False

        groupings = ["HEREPOI", "OSM", "Road", "Terminal"]
        for grouping in groupings:
            keys = [k for k,v in self.getVertexAttrs().items() if v == grouping]            
            for vertexName in self.getVertices():
                vertex = self.getVertexByName(vertexName, 'attr')
                if verydebug:
                    print("  --> Vertex Number {0} and ID {1}".format(vertexNum, vertexID))

                for key in keys:
                    value   = vertex[key]

                    result = None
                    if isinstance(value, list):
                        if value[0][0] is not None:
                            result = True
                        else:
                            result = False
                    else:
                        print("Input {0} is type {1}".format(value, type(value)))

                    self.setVertexFeature(vertexName, key, result)
                    if verydebug is True:
                        print("\t: {0}, {1} == {2}".format(key, value, result))
                    
        if verydebug:
            raise ValueError("Stoppping after verydebug is True")                    
            

    def fillInternalData(self, debug=False):
        if debug:
            print("Filling Vertex Internal Data")
        verydebug=False

        keys = [k for k,v in self.getVertexAttrs().items() if v == "General"]
        for vertexName in self.getVertices():
            vertex = self.getVertexByName(vertexName, 'attr')
            if verydebug:
                print("  --> Vertex Number {0} and ID {1}".format(vertexNum, vertexID))

            for key in keys:
                value   = vertex[key]
                feature = self.categories.getFeatures(key, value, debug)
                if isinstance(feature, dict):
                    feature = feature.get('Name')
                self.setVertexFeature(vertexName, key, feature)
                if verydebug is True:
                    print("\t: {0}, {1} == {2}".format(key, value, feature))
                    
        if verydebug:
            raise ValueError("Stoppping after verydebug is True")                             
            

    def fillEdgeData(self, debug=False):
        if debug:
            print("Filling Edge Data")
        verydebug=False

        for edgeName in self.getEdges():
            features = [self.getVertexByName(x, 'feat') for x in tuple(edgeName)]
            for key,feat1 in features[0].items():
                feat2 = features[1][key]
                value = [str(feat1), str(feat2)]
                self.setEdgeFeature(edgeName, key, value)
                if verydebug is True:
                    print("\t: {0}, {1} == {2}".format(edgeName, key, value))
                    
        if verydebug:
            raise ValueError("Stoppping after verydebug is True")

In [400]:
%load_ext autoreload
%autoreload

from edgeInfo import edgeInfo
from vertexInfo import vertexInfo
from networkCategories import categories

dn = driverNetwork(trips)
dn.create()
dn.computeNetworkAttrs(debug=True)
dn.fillCensusData(debug=True)
dn.fillGeospatialData(debug=True)
dn.fillInternalData(debug=True)
dn.fillEdgeData(debug=True)
g = dn.getNetwork()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Creating a driver network with 21 vertices and 197 edges.


/Users/tgadf/Documents/code/network/networkAlgos.py:460: ComplexWarning: Casting complex values to real discards the imaginary part
  v = float(max(v))


  Created 39 attributes for 21 vertices
  Created 2 attributes for 197 edges
  Created 48 attributes for the network
Filling Vertex Census Data
Filling Vertex Geospatial Data
Filling Vertex Internal Data
Filling Edge Data


In [447]:
class networkFeatures():
    def __init__(self, dn):
        self.dn = dn
        self.features = {}
        
        self.categories        = categories(debug)
        self.getCategories     = self.categories.getCategories
        self.getCategory       = self.categories.getCategory
        self.getPermCategories = self.categories.getPermCategories
        self.getPermCategory   = self.categories.getPermCategory  
        self.getHomeRatioCategory = self.categories.getHomeRatioCategory
        self.getIntervalCategory = self.categories.getIntervalCategory
        
        

    #################################################################################################################
    # Vertex/Edge Counts
    #################################################################################################################
    def fillVertexCounts(self, debug=False):
        if debug:
            print("Filling Vertex Counts")
            
        featureNames = None
            
        from collections import Counter
        vertexCounts = {"N": {}, 3: {}, 10: {}, 25: {}}

        for vertexNum,vertexName in enumerate(self.dn.getVertices()):
            vertex = dn.getVertexByName(vertexName, 'feat')
            if featureNames is None:
                featureNames = list(vertex.keys())
                for featureName in featureNames:
                    featCats = self.getCategories(featureName)
                    if featCats is not None:
                        for cat in featCats:
                            key = "".join([featureName,cat])
                            for cutoff in ["N",3,10,25]:
                                vertexCounts[cutoff][key]  = 0
                    else:
                        for cutoff in ["N",3,10,25]:
                            vertexCounts[cutoff][featureName]  = 0
                        
            
            for featureName in featureNames:                
                value = vertex[featureName]
                featCats = self.getCategories(featureName)
                if featCats is not None and value in featCats:
                    key = "".join([featureName,value])
                    vertexCounts["N"][key] += 1
                    for cutoff in [3,10,25]:
                        if vertexNum < cutoff:
                            vertexCounts[cutoff][key] += 1
                            
                            
        retval = {}
        for cutoff,cutoffData in vertexCounts.items():
            for key,value in cutoffData.items():
                if retval.get(key) is None:
                    retval[key] = {}
                if isinstance(cutoff, int):
                    retval[key]["".join(["Top", str(cutoff)])] = value
                else:
                    retval[key][cutoff] = value
                                
        self.features["Vertex_Counts"] = retval
        
            

    #################################################################################################################
    # Vertex/Edge Properties
    #################################################################################################################
    def fillObjectProperties(self, objectData, debug=False):
        try:
            diffVtx0Vtx1  = float(objectData[0] - objectData[1])
        except:
            diffVtx0Vtx1  = None

        try:
            diffVtx1Vtx2  = float(objectData[1] - objectData[2])
        except:
            diffVtx1Vtx2  = None

        try:
            diffVtx0Vtx12 = float(objectData[0] - objectData[1] - objectData[2])
        except:
            diffVtx0Vtx12 = None

        try:
            qvals = list(objectData.quantile(q=[0.05,0.25,0.5,0.75,0.95]))
        except:
            qvals = [None, None, None, None, None]

        retval = {"Diff_First_Second":  diffVtx0Vtx1,
                  "Diff_Second_Third":  diffVtx1Vtx2,
                  "Diff_Top3":         diffVtx0Vtx12,
                  "Very_Low_Quantile":  qvals[0],
                  "Low_Quantile":      qvals[1],
                  "Mid_Quantile":      qvals[2],
                  "High_Quantile":     qvals[3],
                  "Very_High_Quantile": qvals[4]}
        return retval
        

    def fillVertexProperties(self, debug=False):
        if debug:
            print("Filling Vertex Properties")

        retval = {}
        vertexAttrs = self.dn.nodeAttrs
        for attribute in vertexAttrs.columns:
            vertexData = getColData(vertexAttrs, colnames=attribute)            
            retval[attribute] = self.fillObjectProperties(vertexData)

        self.features["Vertex_Properties"] = retval
        

    def fillEdgeProperties(self, debug=False):
        if debug:
            print("Filling Edge Properties")

        retval = {}
        edgeAttrs = self.dn.edgeAttrs
        for attribute in edgeAttrs.columns:
            edgeData = getColData(edgeAttrs, colnames=attribute)
            retval[attribute] = self.fillObjectProperties(edgeData)

        self.features["Edge_Properties"] = retval

        
        
    #################################################################################################################
    # Top Vertex/Edge Features
    #################################################################################################################
    def fillIndividualVertexFeatures(self, debug=False):
        key = "Vertex_Top5"
        retval = {}
        
        for vertexNum in range(5):
            vertex = dn.getVertex(vertexNum, 'feat')
            retval["{0}".format(vertexNum)] = self.fillIndividualObjectFeatures(vertexNum, vertex, debug=debug)
            
        self.features[key] = retval
        
        
    def fillIndividualEdgeFeatures(self, debug=False):
        key = "Edge_Top5"
        retval = {}
        
        for edgeNum in range(5):
            edge = dn.getEdge(edgeNum, 'feat')
            retval["{0}".format(edgeNum)] = self.fillIndividualObjectFeatures(edgeNum, edge, debug=debug)
            
        self.features[key] = retval
        
        
    def fillIndividualObjectFeatures(self, objectNum, objectData, debug=False):
        retval = {}
        retval['Rank'] = objectNum
        for featureName, featureValue in objectData.items():
            retval[featureName] = featureValue
        return retval
        
        
    def fillNetworkFeatures(self, debug=False):
        key = "Network"
        retval = {}
        
        netAttrs = self.dn.netAttrs
        for featureName, featureValue in netAttrs.items():
            retval[featureName] = featureValue
            
        self.features[key] = retval


    def fillHomeFeatures(self, debug=False):
        key = "Home"
        retval = {}
        
        vertexName = str(dn.homeMetrics['Vtx'])
        vertexData = dn.getVertexByName(vertexName, 'feat')
        vertexNum  = dn.getVertexNum(vertexName)
        retval["Rank"] = vertexNum
        ratio = dn.homeMetrics['Ratio']
        ratio_significance = self.getHomeRatioCategory(ratio, debug)
        retval["Ratio"]    = ratio_significance
        retval["Days"]     = dn.homeMetrics['Days']
        retval["Days"], _  = self.getIntervalCategory(retval["Days"], debug)
        for featureName, featureValue in vertexData.items():
            retval[featureName] = featureValue

        self.features[key] = retval
        
                
        
    #################################################################################################################
    # Feature Correlations
    #################################################################################################################
    def fillFeatureCorrelations(self, debug=False):
        key = "Vertex_Corr"
        retval = {}
        
        vertexAttrs = self.dn.nodeAttrs
        for i,attribute1 in enumerate(vertexAttrs.columns):
            vertexData1 = getColData(vertexAttrs, colnames=attribute1)
            for j,attribute2 in enumerate(vertexAttrs.columns):
                if j <= i:
                    continue
                    
                vertexData2 = getColData(vertexAttrs, colnames=attribute2)               
                try:
                    corr = vertexData1.corr(vertexData2)
                except:
                    corr = None
                retval["_".join([attribute1, attribute2])] = corr

        self.features[key] = retval
        
        key = "Edge_Corr"
        retval = {}
        
        edgeAttrs = self.dn.edgeAttrs
        for i,attribute1 in enumerate(edgeAttrs.columns):
            edgeData1 = getColData(edgeAttrs, colnames=attribute1)
            for j,attribute2 in enumerate(edgeAttrs.columns):
                if j <= i:
                    continue
                    
                edgeData2 = getColData(edgeAttrs, colnames=attribute2)               
                try:
                    corr = edgeData1.corr(edgeData2)
                except:
                    corr = None
                retval["_".join([attribute1, attribute2])] = corr

        self.features[key] = retval
        
        


    

    #######################################################################################################################
    #
    # Create DataFrame
    #
    #######################################################################################################################
    def fixType(self, value):
        import numpy as np
        if isinstance(value, tuple):
            value = str(value)
        elif isinstance(value, np.int64):
            value = int(value)
        elif isinstance(value, np.float64):
            value = float(value)
        elif isinstance(value, str):
            value = str(value)
        elif isinstance(value, float):
            value = float(value)
        elif isinstance(value, int):
            value = int(value)
        elif isinstance(value, type(None)):
            value = None
        else:
            raise ValueError("Unknown Type: {0} --> {1}".format(type(value), value))
        return value
                        
    def getFeatureDataFrame(self, debug=False):
        from pandas import DataFrame
        from collections import Counter
        features = {}
        cntr = Counter()
        for category, categorydata in self.features.items():
            for feature, featuredata in categorydata.items():
                if isinstance(featuredata, dict):
                    for subfeature, subfeaturedata in featuredata.items():
                        key = "_".join([category,feature,subfeature])
                        key = "".join([s.title() for s in key.split("_")])
                        value = self.fixType(subfeaturedata)
                        features[key] = value
                else:
                    key = "_".join([category,feature])
                    key = "".join([s.title() for s in key.split("_")])
                    value = self.fixType(featuredata)
                    features[key] = value
        
        if debug:
            print("Created Data Frame with {0} features".format(len(features)))

        if False:
            features['Device'] = self.device
            if self.expectedFeatures is not None:
                if len(features) != self.expectedFeatures:
                    print("\nThere are only {0}/{1} features for {2}!!!\n".format(len(features), self.expectedFeatures, self.device))
                    self.printFeatures()
                    raise ValueError("\nThere are only {0}/{1} features for {2}!!!\n".format(len(features), self.expectedFeatures, self.device))

        df = DataFrame(features, index=[0])
        return df

In [448]:
nf = networkFeatures(dn)
nf.fillEdgeProperties()
nf.fillVertexCounts()
nf.fillVertexProperties()
#nf.fillIndividualVertexFeatures()
#nf.fillIndividualEdgeFeatures()
nf.fillNetworkFeatures()
nf.fillHomeFeatures()
nf.fillFeatureCorrelations()

In [449]:
nf.getFeatureDataFrame().T

,0
EdgePropertiesEdgeBetweennessCentralityDiffFirstSecond,-0.000132275
EdgePropertiesEdgeBetweennessCentralityDiffSecondThird,0.000146199
EdgePropertiesEdgeBetweennessCentralityDiffTop3,-0.00514481
EdgePropertiesEdgeBetweennessCentralityVeryLowQuantile,0.0047619
EdgePropertiesEdgeBetweennessCentralityLowQuantile,0.00507937
EdgePropertiesEdgeBetweennessCentralityMidQuantile,0.00545936
EdgePropertiesEdgeBetweennessCentralityHighQuantile,0.00639456
EdgePropertiesEdgeBetweennessCentralityVeryHighQuantile,0.00779606
EdgePropertiesEdgeWeightDiffFirstSecond,1
EdgePropertiesEdgeWeightDiffSecondThird,-2


In [440]:
[v,x for v,x in trips["vertexMetrics"].items()]

SyntaxError: invalid syntax (<ipython-input-440-4c448f14f173>, line 1)

# Load Data

# Run the Network

In [333]:
dn.getVertexByName(str(dn.homeMetrics['Vtx']))

{'DayOfWeek': {'Avg': 0.0, 'Std': 0.0},
 'DrivingDistance': {'Avg': 17.9203779245874, 'Std': 35.0725549687412},
 'GeoDistanceRatio': {'Avg': 17.9203779245874, 'Std': 35.0725549687412},
 'N': 338,
 'First': datetime.date(2018, 11, 12),
 'Last': datetime.date(2018, 11, 12),
 'CoM': (29.8353, 49.95551),
 'Geo': ['tj798z2d',
  'tj798z2e',
  'tj798z29',
  'tj798z20',
  'tj798z26',
  'tj798z2k',
  'tj798z27',
  'tj798xpp',
  'tj798z3r',
  'tj798z30',
  'tj798z25',
  'tj798xru',
  'tj798z3p',
  'tj798z2n',
  'tj798z33',
  'tj798z1p',
  'tj798z8c',
  'tj798xrd',
  'tj798z0z',
  'tj798z87',
  'tj798xrf',
  'tj798z2b',
  'tj798z2p',
  'tj798z2m',
  'tj798z2q',
  'tj798xpf',
  'tj798z22',
  'tj798z8q',
  'tj798z8s',
  'tj798xrg',
  'tj798z2h',
  'tj798z2s',
  'tj798z80',
  'tj798xre',
  'tj798z2x',
  'tj798z3j',
  'tj798z0r',
  'tj798z06',
  'tj798z03',
  'tj798xxg',
  'tj798xr2',
  'tj798xr9',
  'tj798z2f',
  'tj798z2r',
  'tj798z38',
  'tj798z9d',
  'tj798z0v',
  'tj798xr4',
  'tj798z24',
  'tj

In [314]:
dn.homeMetrics
dn.

{'Geo': 'cl0',
 'Vtx': 7,
 'Ratio': 3.2,
 'Days': None,
 'Possible': 1,
 'Location': (29.8353, 49.95551)}

In [328]:
[x[1]['CoM'] for x in g.nodes(data=True)]

[(29.92631, 50.06197),
 (29.94292, 49.92098),
 (30.1047, 49.88418),
 (30.10656, 49.86978),
 (30.14662, 49.87287),
 (29.83176, 49.93469),
 (30.14525, 49.95791),
 (29.8353, 49.95551),
 (29.94615, 49.97473),
 (29.90528, 50.07727),
 (29.95516, 49.94439),
 (30.09337, 50.0867),
 (30.00332, 50.04058),
 (30.16217, 49.84926),
 (29.87096, 49.93795),
 (29.89358, 50.13432),
 (30.04229, 50.09493),
 (30.07927, 49.9494),
 (30.1178, 49.86881),
 (30.06101, 50.15843),
 (29.89108, 50.13519)]

In [ ]:
results = runNetworkAlgorithms(g)

In [ ]:
results["Net"]

In [ ]:
#set(a.keys()).intersection(set(b.keys()))
dn.vertexAttrs

In [ ]:
#idx.reindex = list(g.edges())
#df = DataFrame(x['Edges'])
#df.columns.droplevel()
#df.reindex = list(g.edges())
df.reset_index(drop=True)

In [ ]:
nodeAlgos = []
edgeAlgos = []
netAlgos  = []
noneAlgos = []
for k,v in x.items():
    if v is None:
        noneAlgos.append(k)
        continue
    if isinstance(v, dict):
        if len(v) == 20:
            nodeAlgos.append(k)
        elif len(v) == 186:
            edgeAlgos.append(k)
        else:
            netAlgos.append(k)
    else:
        netAlgos.append(k)

In [ ]:
#nodeAlgos
#edgeAlgos
import json
json.dumps(edgeAlgos)
#netAlgos
#runAlgos(algosToRun, g)

In [ ]:
x

In [ ]:

runAlgos(algosToRun, g)

In [ ]:

#runAlgos(algosToRun, g)

In [ ]:

runAlgos(algosToRun, g)

In [ ]:

runAlgos(algosToRun, g)

In [ ]:

runAlgos(algosToRun, g)

In [ ]:

#algosToRun.append(algos.eulerian_circuit)
runAlgos(algosToRun, g)

In [ ]:
#runAlgos(algosToRun, g)

In [ ]:
#runAlgos(algosToRun, g)

In [ ]:
runAlgos(algosToRun, g)

In [ ]:
runAlgos(algosToRun, g)

In [ ]:
algosToRun = []
#algosToRun.append(algos.node_classification.harmonic_function)
#algosToRun.append(algos.node_classification.local_and_global_consistency)
#runAlgos(algosToRun, g)

In [ ]:
runAlgos(algosToRun, g)

In [ ]:
runAlgos(algosToRun, g)

In [ ]:
#runAlgos(algosToRun, g)

In [ ]:
algosToRun = []
#algosToRun.append(algos.sigma)
#algosToRun.append(algos.omega)
#runAlgos(algosToRun, g)

In [ ]:
runAlgos(algosToRun, g)

In [ ]:
#runAlgos(algosToRun, g)

In [ ]:

runAlgos(algosToRun, g)

In [ ]:
runAlgos(algosToRun, g)

In [ ]:

runAlgos(algosToRun, g)

In [ ]:
runAlgos(algosToRun, g)

In [ ]:
runAlgos(algosToRun, g)

In [ ]:
# Get Sparse Matrix
algosToRun = []
algosToRun.append(linalg.attr_sparse_matrix)
#algosToRun.append(convert_matrix.from_scipy_sparse_matrix)
algosToRun.append(convert_matrix.to_pandas_adjacency)
#runAlgos(algosToRun, g)

In [ ]:
convert_matrix.to_pandas_edgelist(g)

In [ ]:
G = nx.complete_graph(5)
A = nx.nx_agraph.to_agraph(G)
H = nx.nx_agraph.from_agraph(A)

In [ ]:
import pygraphviz as pgv
print(A)

In [ ]:
print(nx.draw(g))

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
import copy
import networkx
import matplotlib.pyplot as plt

# Generate a graph.
# Here I chose an ER graph.
g = nx.erdos_renyi_graph(20, 0.3)

# Get positions.
# Here I use the spectral layout and add a little bit of noise.
pos = nx.layout.spectral_layout(g)
pos = nx.spring_layout(g, pos=pos, iterations=50)

# Create position copies for shadows, and shift shadows
pos_shadow = copy.deepcopy(pos)
shift_amount = 0.006
for idx in pos_shadow:
    pos_shadow[idx][0] += shift_amount
    pos_shadow[idx][1] -= shift_amount

#~~~~~~~~~~~~
# Draw graph
#~~~~~~~~~~~~
fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

nx.draw_networkx_nodes(g, pos_shadow, node_color='k', alpha=0.5)
nx.draw_networkx_nodes(g, pos, node_color="#3182bd", linewidths=1)
nx.draw_networkx_edges(g, pos, width=1)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

G = nx.Graph(day="Stackoverflow")
df_nodes = pd.read_csv('../input/stack_network_nodes.csv')
df_edges = pd.read_csv('../input/stack_network_links.csv')

for index, row in df_nodes.iterrows():
    G.add_node(row['name'], group=row['group'], nodesize=row['nodesize'])
    
for index, row in df_edges.iterrows():
    G.add_weighted_edges_from([(row['source'], row['target'], row['value'])])
    
color_map = {1:'#f09494', 2:'#eebcbc', 3:'#72bbd0', 4:'#91f0a1', 5:'#629fff', 6:'#bcc2f2',  
             7:'#eebcbc', 8:'#f1f0c0', 9:'#d2ffe7', 10:'#caf3a6', 11:'#ffdf55', 12:'#ef77aa', 
             13:'#d6dcff', 14:'#d2f5f0'} 

plt.figure(figsize=(25,25))
options = {
    'edge_color': '#FFDEA2',
    'width': 1,
    'with_labels': True,
    'font_weight': 'regular',
}
colors = [color_map[G.node[node]['group']] for node in G]
sizes = [G.node[node]['nodesize']*10 for node in G]

"""
Using the spring layout : 
- k controls the distance between the nodes and varies between 0 and 1
- iterations is the number of times simulated annealing is run
default k=0.1 and iterations=50
"""
nx.draw(G, node_color=colors, node_size=sizes, pos=nx.spring_layout(G, k=0.25, iterations=50), **options)
ax = plt.gca()
ax.collections[0].set_edgecolor("#555555") 
plt.show()

In [ ]:
dn.edgeDict[('0', '1')].values()

In [ ]:

    minmaxWeight = [0.0, 2.5]
    print("Number of Edges: {0}".format(nEdges))
    nRange=5
    if nEdges > 100000:
        minmaxWeight[1] = 2
        nRange=6
        weightSize = [power(x,11) for x in linspace(minmaxWeight[0], minmaxWeight[1], nRange)]
    elif nEdges > 50000:
        minmaxWeight[1] = 2
        weightSize = [power(x,9) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 25000:
        weightSize = [power(x,8) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 10000:
        weightSize = [power(x,7) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 2000:
        weightSize = [power(x,6) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 1000:
        weightSize = [power(x,5) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 500:
        weightSize = [power(x,4) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 100:
        weightSize = [power(x,3) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    else:
        weightSize = [power(x,2) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    scale = 2.5/amax(weightSize)
    weightSize = [x*scale for x in weightSize]

In [ ]:
for k,k2,v in g.edges(data=True):
    print(v)
    break
#self.nodeDict = {u: d for (u,d) in self.g.nodes(data=True)}


In [162]:
from collections import Counter
x = Counter()
x[3] += 1
x

Counter({3: 1})

In [164]:
x.get(4)

In [252]:
from pandas import Series
tmp = Series([1, 3, 45,6 ,8, 34])

In [255]:
list(tmp.quantile(q=[0.05,0.95]))

[1.5, 42.25]

In [266]:
g.edges(data=True)

EdgeDataView([('0', '1', {'Duration': {'Avg': 0.0, 'Std': 0.0}, 'First': datetime.date(2018, 11, 12), 'Last': datetime.date(2018, 11, 12), 'DayOfWeek': {'Avg': 0.0, 'Std': 0.0}, 'ITA': {'Avg': 0.0, 'Std': 0.0}, 'GeoDistance': {'Avg': 13.851113564761874, 'Std': 0.10604816108480218}, 'DrivingDistance': {'Avg': 22.291151104313872, 'Std': 0.1706675475602148}, 'GeoDistanceRatio': {'Avg': 22.291151104313872, 'Std': 0.1706675475602148}, 'Weight': 4, 'Geos': ['cl14', 'cl4'], 'Locations': [(29.92631, 50.06197), (29.94292, 49.92098)], 'Interval': 1, 'FractionalActive': 1.0}), ('0', '3', {'Duration': {'Avg': 0.0, 'Std': 0.0}, 'First': datetime.date(2018, 11, 12), 'Last': datetime.date(2018, 11, 12), 'DayOfWeek': {'Avg': 0.0, 'Std': 0.0}, 'ITA': {'Avg': 0.0, 'Std': 0.0}, 'GeoDistance': {'Avg': 27.234485944146744, 'Std': 0.18733019423077532}, 'DrivingDistance': {'Avg': 43.82954760935312, 'Std': 0.30147797478335636}, 'GeoDistanceRatio': {'Avg': 43.82954760935312, 'Std': 0.30147797478335636}, 'Weight

In [451]:
#test

,device,Start,End,Duration,total_miles,inR4HTimePeriod,lat0,long0,lat1,long1,...,Geo0ROADSMajorRdID,Geo1ROADSMajorRdID,Geo0POIVisitsID,Geo0POIUniqueVisitsID,Geo1POIVisitsID,Geo1POIUniqueVisitsID,Geo0TerminalsAirportID,Geo0TerminalsAmtrakID,Geo1TerminalsAirportID,Geo1TerminalsAmtrakID
0,RWDEB97LSA1H4IZ,2017-12-23T10:01:48-10:00,2017-12-23T10:23:44-10:00,1315,14.10,Y,21.385278,-157.950076,21.312765,-158.055588,...,0,0,1853,191,0,0,0.0,0.0,0.0,0.0
1,53XC3IYW67C4O0E,2017-10-31T15:44:31-10:00,2017-10-31T17:08:54-10:00,5063,13.20,Y,21.440997,-158.184695,21.339682,-158.078461,...,0,0,6,3,1263,54,0.0,0.0,0.0,0.0
2,M4CGPR1TYHBKVJK,2017-12-24T03:23:26-10:00,2017-12-24T03:39:59-10:00,992,8.40,Y,21.293925,-157.821295,21.334003,-157.923459,...,1,0,48,20,199,97,0.0,0.0,1.0,0.0
3,RWDEB97LSA1H4IZ,2017-12-31T16:16:26-10:00,2017-12-31T16:26:32-10:00,605,3.85,Y,21.330069,-157.879447,21.333042,-157.923245,...,0,0,225,14,199,97,0.0,0.0,1.0,0.0
4,3T96FV77UYC17DJ,2017-12-22T18:02:46-10:00,2017-12-22T18:33:13-10:00,1827,7.80,Y,21.367507,-157.925737,21.327959,-157.899446,...,0,0,4,1,4,1,0.0,0.0,0.0,0.0
5,PFLXOYIQSN535UC,2017-12-25T10:05:32-10:00,2017-12-25T10:30:09-10:00,1476,12.64,Y,21.275205,-157.824777,21.328160,-157.899595,...,1,0,456,56,4,1,0.0,0.0,0.0,0.0
6,JY2R26AMISS5BG1,2017-12-28T20:27:50-10:00,2017-12-28T20:37:43-10:00,593,5.73,Y,21.278178,-157.811567,21.321931,-157.866710,...,1,0,13,4,542,78,0.0,0.0,0.0,0.0
7,BZU6BCY6DYBNOFY,2017-07-17T15:04:16-10:00,2017-07-17T15:23:10-10:00,1134,5.41,Y,21.278456,-157.811646,21.321742,-157.856744,...,1,0,13,4,840,131,0.0,0.0,0.0,0.0
8,NCGFHU5YH0N8BXA,2017-09-07T12:50:37-10:00,2017-09-07T13:01:18-10:00,640,3.30,Y,21.359389,-157.853660,21.322711,-157.856991,...,0,0,4,2,840,131,0.0,0.0,0.0,0.0
9,BZU6BCY6DYBNOFY,2017-09-05T09:20:29-10:00,2017-09-05T09:37:34-10:00,1024,4.07,Y,21.298647,-157.840693,21.322458,-157.857047,...,0,0,2012,129,840,131,0.0,0.0,0.0,0.0
